# Hacker News

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

* id: The unique identifier from Hacker News for the post
* title: The title of the post
* url: The URL that the posts links to, if it the post has a URL
* num_points: The number of points the post acquired, calculated as the * * total number of upvotes minus the total number of downvotes
* num_comments: The number of comments that were made on the post
* author: The username of the person who submitted the post
* created_at: The date and time at which the post was submitted

Let's start by importing the libraries we need and reading the data set into a list of lists.

In [2]:
from csv import reader
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]

print(headers)                #Printing Header
for i in hn[:5]:                #Printing first five rows
    print(i)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


In [11]:
ask_posts = []
show_posts = []
other_posts = []

for i in hn:
    title = i[1]
    title = title.lower()
    if(title.startswith('ask hn')):
        ask_posts.append(i)
    elif(title.startswith('show hn')):
        show_posts.append(i)
    else:
        other_posts.append(i)

print('length of ask_posts = {}\nlength of show_posts = {}\nlenght of other_posts = {}'.format(len(ask_posts),len(show_posts),len(other_posts)))

length of ask_posts = 1744
length of show_posts = 1162
lenght of other_posts = 17194



In the last screen, we separated the "ask posts" and the "show posts" into two list of lists named ask_posts and show_posts. Below are the first five rows in the ask_posts list of lists:


In [8]:
for i in ask_posts[:5]:
    print(i)

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']
['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']
['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']
['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']
['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']


In [32]:
for i in show_posts[:5]:
    print(i)

['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']
['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']
['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05']
['12178806', 'Show HN: Webscope  Easy way for web developers to communicate with Clients', 'http://webscopeapp.com', '3', '3', 'fastbrick', '7/28/2016 7:11']
['10872799', 'Show HN: GeoScreenshot  Easily test Geo-IP based web pages', 'https://www.geoscreenshot.com/', '1', '9', 'kpsychwave', '1/9/2016 20:45']


# Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [14]:
total_ask_comments = 0
for i in ask_posts:
    total_ask_comments += (int)(i[4])
avg_ask_comments = total_ask_comments / len(ask_posts)
print('average number of comments on ask posts = {}'.format(avg_ask_comments))

total_show_comments = 0
for i in show_posts:
    total_show_comments += (int)(i[4])
avg_show_comments = total_show_comments / len(show_posts)
print('average number of comments on show posts = {}'.format(avg_show_comments))

average number of comments on ask posts = 14.038417431192661
average number of comments on show posts = 10.31669535283993


Form above claculations, we can observe that ask posts (14.0384) receive more comments on average as show posts (10.3167).

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

# Finding the Amount of Ask Posts and Comments by Hour Created¶

In [26]:
import datetime as dt
result_list = []
for i in ask_posts:
    x = []
    x.append(i[6])
    x.append((int)(i[4]))
    result_list.append(x)

counts_by_hour = {}
comments_by_hour = {}
    
for i in result_list:
    hour = dt.datetime.strptime(i[0], "%m/%d/%Y %H:%M")
    hour = dt.datetime.strftime(hour, "%H")
    if(hour not in counts_by_hour):
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = i[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += i[1]

print(counts_by_hour)
print(comments_by_hour)

{'12': 73, '03': 54, '00': 55, '11': 58, '19': 110, '18': 109, '01': 60, '17': 100, '04': 47, '08': 48, '09': 45, '15': 116, '13': 85, '16': 108, '23': 68, '21': 109, '20': 80, '02': 58, '07': 34, '05': 46, '10': 59, '22': 71, '06': 44, '14': 107}
{'12': 687, '03': 421, '00': 447, '11': 641, '19': 1188, '18': 1439, '01': 683, '17': 1146, '04': 337, '08': 492, '09': 251, '15': 4477, '13': 1253, '16': 1814, '23': 543, '21': 1745, '20': 1722, '02': 1381, '07': 267, '05': 464, '10': 793, '22': 479, '06': 397, '14': 1416}



Let's use this format to create a list of lists containing the hours during which posts were created and the average number of comments those posts received.



In [30]:
avg_by_hour = []

for i in counts_by_hour:
    x = []
    x.append(i)
    x.append(comments_by_hour[i] / counts_by_hour[i])
    avg_by_hour.append(x)
    
for i in avg_by_hour:
    print(i)

['12', 9.41095890410959]
['03', 7.796296296296297]
['00', 8.127272727272727]
['11', 11.051724137931034]
['19', 10.8]
['18', 13.20183486238532]
['01', 11.383333333333333]
['17', 11.46]
['04', 7.170212765957447]
['08', 10.25]
['09', 5.5777777777777775]
['15', 38.5948275862069]
['13', 14.741176470588234]
['16', 16.796296296296298]
['23', 7.985294117647059]
['21', 16.009174311926607]
['20', 21.525]
['02', 23.810344827586206]
['07', 7.852941176470588]
['05', 10.08695652173913]
['10', 13.440677966101696]
['22', 6.746478873239437]
['06', 9.022727272727273]
['14', 13.233644859813085]


# Sorting and Printing Values from a List of Lists¶

In [28]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[9.41095890410959, '12'], [7.796296296296297, '03'], [8.127272727272727, '00'], [11.051724137931034, '11'], [10.8, '19'], [13.20183486238532, '18'], [11.383333333333333, '01'], [11.46, '17'], [7.170212765957447, '04'], [10.25, '08'], [5.5777777777777775, '09'], [38.5948275862069, '15'], [14.741176470588234, '13'], [16.796296296296298, '16'], [7.985294117647059, '23'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [7.852941176470588, '07'], [10.08695652173913, '05'], [13.440677966101696, '10'], [6.746478873239437, '22'], [9.022727272727273, '06'], [13.233644859813085, '14']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [29]:
# Sort the values and print the the 5 hours with the highest average comments.

print("Top 5 Hours for 'Ask HN' Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )

Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion
In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.